In [7]:
from pathlib import Path
import pandas as pd
import os, json
from analysis_utils import get_tuned_alg_perf

In [8]:
def read_json(file):
    with open(file, 'r') as f:
        return json.load(f)

In [2]:
metadata_folder = Path("../TabZilla")
metadataset_df = pd.read_csv(metadata_folder / "tutorials/metadataset_clean.csv")

In [3]:
metadataset_df.head()

,dataset_fold_id,dataset_name,target_type,alg_name,hparam_source,Log Loss__train,Log Loss__val,Log Loss__test,AUC__train,AUC__val,...,Accuracy__train,Accuracy__val,Accuracy__test,F1__train,F1__val,F1__test,training_time,eval-time__train,eval-time__val,eval-time__test
0,openml__APSFailure__168868__fold_0,openml__APSFailure__168868,binary,CatBoost,default,0.018501,0.026880,0.020194,0.990573,0.986760,...,0.994523,0.991053,0.993421,0.994523,0.991053,0.993421,4.039096,0.185535,0.028121,0.034166
1,openml__APSFailure__168868__fold_0,openml__APSFailure__168868,binary,CatBoost,random_1_s0,0.011968,0.024195,0.017324,0.995348,0.988589,...,0.997533,0.991974,0.994605,0.997533,0.991974,0.994605,3.624039,0.129900,0.020647,0.017871
2,openml__APSFailure__168868__fold_0,openml__APSFailure__168868,binary,CatBoost,random_10_s0,0.022589,0.029668,0.023737,0.990179,0.985547,...,0.992566,0.989605,0.990921,0.992566,0.989605,0.990921,2.494685,0.074099,0.021543,0.019580
3,openml__APSFailure__168868__fold_0,openml__APSFailure__168868,binary,CatBoost,random_11_s0,0.028515,0.034531,0.028539,0.987448,0.985578,...,0.990641,0.986974,0.990132,0.990641,0.986974,0.990132,2.532277,0.061874,0.014086,0.013869
4,openml__APSFailure__168868__fold_0,openml__APSFailure__168868,binary,CatBoost,random_12_s0,0.015119,0.025570,0.019329,0.993757,0.989098,...,0.996382,0.991447,0.993684,0.996382,0.991447,0.993684,4.220030,0.121606,0.034948,0.033927


In [ ]:
alldatasets = metadataset_df["dataset"].unique()

In [6]:
metadataset_df.columns

Index(['dataset_fold_id', 'dataset_name', 'target_type', 'alg_name',
       'hparam_source', 'Log Loss__train', 'Log Loss__val', 'Log Loss__test',
       'AUC__train', 'AUC__val', 'AUC__test', 'Accuracy__train',
       'Accuracy__val', 'Accuracy__test', 'F1__train', 'F1__val', 'F1__test',
       'training_time', 'eval-time__train', 'eval-time__val',
       'eval-time__test'],
      dtype='object')

In [11]:
new_df = []
datasets = os.listdir('../results/TabFlexModel')
for dataset in datasets:
    result = read_json(f'../results/TabFlexModel/{dataset}/default_trial0_results.json')
    for fold_id in range(10):
        new_df.append({
            'dataset_fold_id': f'{dataset}__fold_{fold_id}',
            'dataset_name': dataset,
            'target_type': result['target_type'],
            'alg_name': 'TabFlexModel',
            'hparam_source': result['hparam_source'],
            'Log Loss__train': result['scorers']['train']['Log Loss'][fold_id],
            'Log Loss__val': result['scorers']['val']['Log Loss'][fold_id],
            'Log Loss__test': result['scorers']['test']['Log Loss'][fold_id],
            'AUC__train': result['scorers']['train']['AUC'][fold_id],
            'AUC__val': result['scorers']['val']['AUC'][fold_id],
            'AUC__test': result['scorers']['test']['AUC'][fold_id],
            'Accuracy__train': result['scorers']['train']['Accuracy'][fold_id],
            'Accuracy__val': result['scorers']['val']['Accuracy'][fold_id],
            'Accuracy__test': result['scorers']['test']['Accuracy'][fold_id],
            'F1__train': result['scorers']['train']['F1'][fold_id],
            'F1__val': result['scorers']['val']['F1'][fold_id],
            'F1__test': result['scorers']['test']['F1'][fold_id],
            'training_time': result['timers']['train'][fold_id],
            'time__val': result['timers']['val'][fold_id],
            'time__test': result['timers']['test'][fold_id],
            'eval-time__train': result['timers']['eval-time__train'][fold_id],
            'eval-time__val': .001,
            'eval-time__test': .001,
        })

In [21]:
result['timers']['train-eval']

[26.963222499999997,
 22.861702699999995,
 24.255638599999997,
 22.198761400000024,
 24.42996290000002,
 21.180934000000036,
 24.683547299999987,
 20.123791400000073,
 20.44814770000005,
 27.638040000000046]